In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections as mc

import os
import time
import importlib as imp

import flow_fields as flow
imp.reload(flow)  # Updates any customizations you've made to the module

# Set matplotlib format
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Load Data and Metadata

In [ ]:
# Load preprocessed 4D-STEM data and manually add metadata
path = '../data/4DSTEM/'
filename = 'sample_preprocessed_data_80x80_ss=10_cl=480__q 1.00 3.00 0.05 a 5.00.npy'

q_min, q_max, q_step = (1.0, 3.0, 0.05)
angle_step = 5
step_size = 10  # nm.  This is the distance separating each diffraction pattern in the 4D-STEM scan.  

data = np.load(path + filename)

# Choose Integration Range

In [ ]:
# Preview Peak Positions
aggregate_plot, difference_plot = flow.preview_intensity_vs_q(data, q_min, q_max, q_step)

In [ ]:
# Use the peak preview to choose the integration range for your peaks.

# Note that there are two convoluted peaks represented in the preview, and we only want one of them.  

peak_start = 1.6 
peak_end = 1.8  # Inclusive

integrated_intensity = flow.integrate_peaks(data, q_min, q_max, q_step, peak_start, peak_end)
prepped_intensity = flow.format_flow_inputs(integrated_intensity, rotate=True)

# Create Flow Plot

In [ ]:
peak_width = 40  # degrees
peaks_per_site = 1.2  # This is manually set by the user by viewing the peaks present in the diffraction patterns.  
    
peak_positions = flow.select_peaks(prepped_intensity, peak_width, peaks_per_site)

In [ ]:
# Create line seeds at each peak
seed_density = 2
line_seeds = flow.seed_lines(peak_matrix, step_size, seed_density=seed_density)

# Extend line seeds to create full lines
bend_tolerance = 20
curve_resolution = 2
propagated_lines = flow.propagate_lines(line_seeds, peak_matrix, step_size, bend_tolerance,
                                   curve_resolution=curve_resolution, max_grid_length=100)

# Show a preview, using a subset of the propagated lines
preview_sparsity = 20
propagated_image = flow.plot_solid_lines(propagated_lines, min_length=2, sparsity=preview_sparsity)
plt.show()

# Thin out lines, reducing overlap between lines and creating a more homogeneous line density.  This prevents the
# illusion of high density in regions with good alignment, and makes the image more readable.
line_spacing = 1
spacing_resolution = 10
angle_spacing_degrees = 10
max_overlap_fraction = 0.5
trimmed_lines = flow.trim_lines(propagated_lines, prepped_intensity_matrix.shape, step_size,
                                line_spacing, spacing_resolution, angle_spacing_degrees,
                                max_overlap_fraction=max_overlap_fraction, min_length=5, verbose=False)
trimmed_image = flow.plot_solid_lines(trimmed_lines)
plt.show()

# Add intensity data to lines
line_data = flow.prepare_line_data(trimmed_lines, prepped_intensity_matrix, step_size)
angle_data = line_data[2, :, :]
intensity_data = np.array(line_data[4, :, :])
n_dims, max_length, n_lines = line_data.shape

# Format and Save

In [ ]:
# There are many ways to format the plots.  I suggest keeping settings organized in the format below.

format_codes = [0, 1, 2, 3]
formatted_plots = []
contrast = 3
brightness = 1
gamma = 1
for i, format_code in enumerate(format_codes):
    if format_code == 0:
        # Constant color, linewidth, and alpha
        r, g, b = flow.color_by_angle(np.zeros((max_length, n_lines)))
        linewidth = np.ones((max_length, n_lines)) * 1
        alpha = np.ones((max_length, n_lines))
    elif format_code == 1:
        # Color by angle, alpha by intensity
        alpha = flow.scale_values(flow.smooth(intensity_data, 9), contrast=contrast, gamma=gamma, brightness=brightness)
        linewidth = np.ones((max_length, n_lines)) * 3
        r, g, b = flow.color_by_angle(angle_data)
    elif format_code == 2:
        # Solid Color, alpha by intensity
        r, g, b = flow.color_by_angle(np.zeros((max_length, n_lines)))
        alpha = flow.scale_values(flow.smooth(intensity_data, 9), contrast=contrast, gamma=gamma, brightness=brightness)
        linewidth = np.ones((max_length, n_lines)) * 2
    elif format_code == 3:
        # Faint lines.  Good for overlays.  
        # Solid Color, alpha by intensity
        shape = np.ones((max_length, n_lines))
        r = shape * 0.95
        g = shape * 0.95
        b = shape * 0.95
        alpha = flow.scale_values(flow.smooth(intensity_data, 9), contrast=contrast, gamma=gamma, brightness=brightness)
        linewidth = np.ones((max_length, n_lines)) * 2
        

    flow.plot_graded_lines(trimmed_lines, r, g, b, alpha, linewidth)
    formatted_plots.append(plt.gcf())
    plt.show()

In [ ]:
base_filename = "../figures/sample_3_"

# Save Images
images_to_save = [
    ('propagated_image', propagated_image),
    ('trimmed_image', trimmed_image),
    ('flow_solid', formatted_plots[1]),
    ('flow_color', formatted_plots[2])
]
for name, image in images_to_save:
    image.savefig(base_filename + name + '.png')
    
formatted_plots[3].savefig(base_filename + 'flow_overlay' + '.png', transparent=True)